<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/NLP/NLP-2025/New/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%96%E2%80%AF8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Практическая работа № 8
**Тема.** Проектирование и реализация предметно-специфичных бенчмарков для оценки крупных языковых моделей

---

## 1. Цель и задачи работы

Современные крупные языковые модели (LLM) демонстрируют впечатляющие результаты на **общих бенчмарках** (MMLU, HELM, Big-Bench), однако их эффективность в **узких предметных областях** — образовании, медицине, юриспруденции, локальных языках — часто остаётся неизученной. Стандартные оценки не отражают способность модели работать с **формализованными правилами**, **отраслевой терминологией**, **локальными нормами** или **специфическими форматами заданий**.

**Цель настоящей работы** — сформировать у обучающегося **методологические и практические навыки проектирования, реализации и публикации специализированных бенчмарков**, ориентированных на конкретный домен. Такой бенчмарк позволяет объективно оценить не «общий интеллект» модели, а её **практическую применимость в реальных задачах**.

В качестве **иллюстративного кейса** в работе используется **оценка LLM на задачах основного государственного экзамена (ОГЭ)** по русскому языку и математике — не как единственная цель, а как пример домена с чёткими правилами, структурированными критериями и социальной значимостью. Однако разработанный конвейер применим и к другим сценариям:  
- медицинская диагностика (на основе клинических протоколов),  
- юридическое консультирование (по Гражданскому кодексу РФ),  
- обработка запросов на низкоресурсных языках (татарском, башкирском и др.).

**Основные задачи работы:**

1. Изучить **принципы проектирования бенчмарков**: таксономия задач, выбор метрик, обеспечение воспроизводимости.  
2. Выбрать **предметную область** для бенчмарка (ОГЭ — как один из вариантов).  
3. Собрать и структурировать **корпус заданий и эталонных ответов** с аннотацией по критериям.  
4. Реализовать **конвейер автоматической оценки**, поддерживающий разные типы ответов:  
   - краткие (число, слово, выбор),  
   - развёрнутые (решение, рассуждение, эссе).  
5. Провести **системное тестирование open-weight LLM** (Llama 3, Mistral, Qwen, DeepSeek, ruT5) и, при наличии доступа, коммерческих моделей (GigaChat).  
6. Интегрировать **RAG-подход** (из Работы № 6) для предоставления моделям доступа к **доменной базе знаний** (справочники, кодексы, критерии).  
7. Оценить модели не только по **точности**, но и по **фактической согласованности**, **объяснимости решений** и **соответствию доменным стандартам**.  
8. Проанализировать **разрыв между общими и специализированными бенчмарками**.  
9. Опубликовать бенчмарк в открытом формате и разработать веб-интерфейс для тестирования.

---

## 2. Теоретические предпосылки

### 2.1. Зачем нужны специализированные бенчмарки?

Общие бенчмарки измеряют **широкие способности**: знание фактов, логику, языковую беглость. Однако в прикладных задачах критически важны **узкие компетенции**:

- **Формальная точность**: соблюдение орфографических правил, математических формул, юридических норм.  
- **Структурное соответствие**: решение должно включать определённые шаги или разделы.  
- **Контекстная адекватность**: ответ должен учитывать локальные реалии (например, законодательство РФ или школьную программу).

Специализированный бенчмарк — это **мост между исследовательской моделью и её реальным применением**.

### 2.2. Архитектура бенчмарка

Любой качественный бенчмарк включает:

1. **Корпус заданий** — с чёткой таксономией (предмет, тип, сложность).  
2. **Эталонные ответы** — возможно, несколько вариантов (для развёрнутых задач).  
3. **Критерии оценки** — машинно-интерпретируемые правила или размеченные примеры.  
4. **Конвейер оценки** — автоматизированная система сопоставления ответа модели с эталоном.  
5. **Метрики** — адаптированные под тип задачи (точность, полнота, согласованность).

### 2.3. Типы доменов (примеры)

| Домен | Источник данных | Особенности оценки |
|------|----------------|-------------------|
| **Образование (ОГЭ/ЕГЭ)** | Демоверсии ФИПИ | Строгие критерии, формальные правила |
| **Медицина** | Клинические протоколы, MIMIC | Фактическая точность, безопасность |
| **Юриспруденция** | Кодексы РФ, судебная практика | Соответствие закону, ссылки на статьи |
| **Низкоресурсные языки** | Татарские/башкирские тексты | Поддержка морфологии, орфографии |

> **Важно**: ОГЭ в этой работе — **не цель, а методологический пример**, иллюстрирующий, как строить бенчмарк в домене с чёткими формальными правилами.

---

## 3. Методика и порядок выполнения работы

### 3.1. Этап 1. Выбор домена и сбор корпуса

- **Опция A (ОГЭ)**: сбор заданий по русскому языку и математике из открытых источников (ФИПИ, банк заданий).  
- **Опция B (другой домен)**: например, юридические вопросы по ГК РФ, медицинские кейсы, татарские лингвистические задачи.  
- **Требования к корпусу**:  
  - ≥300 заданий;  
  - разнообразие типов (краткие/развёрнутые);  
  - наличие эталонных ответов и критериев.

### 3.2. Этап 2. Таксономия и аннотация

- Разработка схемы классификации заданий (предмет → тип → сложность).  
- Аннотация эталонных ответов:  
  - для кратких — точный ответ;  
  - для развёрнутых — ключевые элементы («должно содержать: определение, пример, вывод»).

### 3.3. Этап 3. Конвейер оценки

Реализация модуля `domain_evaluator.py`:

- **Краткие ответы**: нормализация + точное совпадение или численная близость.  
- **Развёрнутые ответы**:  
  - **Правило-based**: наличие ключевых фраз/шагов (регулярки, NER);  
  - **Модель-based**: BERTScore против эталона;  
  - **Классификатор**: fine-tuned RuBERT для оценки по критериям (на основе Работы № 4).

### 3.4. Этап 4. Тестирование моделей

- **Open-weight LLM**: Llama 3, Mistral, Qwen, DeepSeek, ruT5;  
- **Режимы**:  
  - Zero-shot,  
  - Few-shot (2–3 примера),  
  - RAG (с доступом к доменной базе знаний).  
- Для ОГЭ: RAG-база = справочник правил и критериев ФИПИ.

### 3.5. Этап 5. Оценка и анализ

- **Метрики**:  
  - Accuracy (краткие),  
  - BERTScore + Coverage (развёрнутые),  
  - Hallucination Rate,  
  - Соответствие критериям (0–100%).  
- **Сравнение**: общие бенчмарки vs ваш специализированный.

### 3.6. Этап 6. Публикация

- Формат: совместим с **EleutherAI LM Evaluation Harness**;  
- Публикация: GitHub + Hugging Face Datasets;  
- Документация: таксономия, метрики, лицензия.

### 3.7. Этап 7. Веб-интерфейс

- Выбор домена и типа задания;  
- Тестирование модели;  
- Отображение: ответ, оценка, критерии, источники (для RAG).

---

## 4. Дополнительные задания

1. **Мультидоменный бенчмарк**: объединить ОГЭ, юридические и медицинские задачи в один набор.  
2. **Генерация заданий**: fine-tune модель на создании новых заданий по теме.  
3. **Оценка на татарском**: адаптация бенчмарка для низкоресурсного языка.  
4. **Сравнение RAG и fine-tuning**: что лучше работает в узком домене?

---

## 5. Требования к отчёту

Отчёт по **ГОСТ 7.32–2017** должен включать:

1. Обоснование выбора домена (ОГЭ — как пример);  
2. Описание таксономии и конвейера оценки;  
3. Результаты тестирования моделей;  
4. Анализ разрыва между общими и специализированными бенчмарками;  
5. Ссылки на публикацию бенчмарка;  
6. Рефлексию: насколько методология применима к другим доменам?

---

## 6. Критерии оценивания

| Оценка | Критерии |
|--------|----------|
| **Отлично (5)** | Полноценный бенчмарк с таксономией, автоматической оценкой, тестированием ≥3 моделей, RAG, публикацией. Чётко показано, что ОГЭ — пример, а не цель. |
| **Хорошо (4)** | Работает конвейер оценки, протестированы модели, есть анализ. |
| **Удовлетворительно (3)** | Собран корпус и реализована базовая оценка (точное совпадение). |
| **Неудовлетворительно (2)** | Отсутствует системный подход к проектированию бенчмарка. |
